In [74]:
import visdom
import numpy as np

v = visdom.Visdom()

assert v.check_connection()

print('visdom is up')

visdom is up


In [75]:
print('initiating visdom windows..')
windows = {
'stat1':{
    'type':'bar',
    'data':[1, 2, 1], # init with 3 unstacked bars
 },
 'stat2':   {
    'type':'line',
    'shape':(50,2),
 },
    'stat3':   {
    'type':'pie',
    'data':[4,13,10] # init with X sections
 },
}

for win_name in windows:
    win = windows[win_name]
    if win['type'] is 'bar':
        v.bar(win['data'], win=win_name)
    
    if win['type'] is 'line':
        v.line(np.zeros(shape=win['shape']), win=win_name)
    
    if win['type'] is 'pie':
        v.pie(win['data'], win=win_name)
    
print('→ done')

initiating visdom windows..
→ done


In [76]:
# basic graphs: bar, line, chart

# draw 3 stacked bars in stat1 window
v.bar([(1,2),(3,2),(1,5)], win=windows['stat1'])

# draw 2 stacked bars in new window
v.bar([(1,2),(3,2)])

# draw pie chart in stat3 window
v.pie(X=[10,10,32,10,10,32], win=windows['stat3'])
         
# update

"{'type': 'pie', 'data': [4, 13, 10]}"

In [209]:
# basic line graph (with no updates)
# visdom line requires `ndim()` to be accessible

# this will work because numpy array has `ndim` attribute
line_y = np.random.rand(10)
print('line_y dimension: {}. type: {}'.format(line_y.ndim, type(line_y)))
v.line(Y=line_y, opts=dict(showlegend=True))

# this will not work because built-in array has no `ndim` attribuet
try:
    line_y = [1,2,3]
    print('line_y dimension: {}. type: {}'.format(line_y.ndim, type(line_y)))
    v.line(Y=line_y, opts=dict(showlegend=True))
except AttributeError as e:
    print('error creating visdom line:', e)

# wrap the array inside numpy array to make above scenario works
line_y = np.array(line_y)
print('line_y dimension: {}. type: {}'.format(line_y.ndim, type(line_y)))
v.line(Y=line_y, opts=dict(showlegend=True))

line_y dimension: 1. type: <class 'numpy.ndarray'>
error creating visdom line: 'list' object has no attribute 'ndim'
line_y dimension: 1. type: <class 'numpy.ndarray'>


'window_3629061dba6d6e'

In [210]:
# draw line graph and push update every 0.5 second

import time

# start with x at [0, 1], and y at [0, 2]
line_x = np.array([0, 1])
line_y = np.array([0, 2])

print('line x: {}, dimension: {}, shape: {}\n{}'.format(type(line_x), 
                                                                     line_x.ndim, 
                                                                     line_x.shape, 
                                                                     line_x))
print('line y: {}, dimension: {}, shape: {}\n{}'.format(type(line_y), 
                                             line_y.ndim, 
                                             line_y.shape,
                                             line_y))
win = v.line(X=line_x, 
             Y=line_y, 
             name='initial state')

epoch = 15
new_y = line_y[-1]
latest_x = line_x[-1]
name = 'line updates'
for i in range(latest_x, latest_x + epoch):
    time.sleep(0.5)
    latest_x = i
    
    if i % 2 == 0:
        new_y -= 1
    elif i % 3 == 0:
        new_y += 1
    elif i % 5 == 0:
        new_y += 5

    print('adding x:{}, y:{}'.format(i, new_y))
    v.line(
        X=np.array([i]),
        Y=np.array([new_y]),
        win=win,
        name=name,
        update='append',
    )

print('stabilizing y to 0 for x at', latest_x+1)
time.sleep(0.5)
v.line(
    X=np.array([latest_x, latest_x+1]),
    Y=np.array([new_y, 0]),
    win=win,
    name='stabilize y to 0',
    update='append',
)

line x: <class 'numpy.ndarray'>, dimension: 1, shape: (2,)
[0 1]
line y: <class 'numpy.ndarray'>, dimension: 1, shape: (2,)
[0 2]
adding x:1, y:2
adding x:2, y:1
adding x:3, y:2
adding x:4, y:1
adding x:5, y:6
adding x:6, y:5
adding x:7, y:5
adding x:8, y:4
adding x:9, y:5
adding x:10, y:4
adding x:11, y:4
adding x:12, y:3
adding x:13, y:3
adding x:14, y:2
adding x:15, y:3
stabilizing y to 0 for x at 16


'window_36290620916a9c'

In [141]:
# more graphs: histogram, contour, etc

# contour
x = np.tile(np.arange(1, 101), (100, 1))
y = x.transpose()
X = np.exp((((x - 50) ** 2) + ((y - 50) ** 2)) / -(20.0 ** 2))
v.contour(X=X, opts=dict(colormap='Viridis'))

'window_362900c87a86b0'

In [142]:
# drawing a surface

# create range
a_range = np.arange(1, 6)
print('create array from int range 1 to 5: {}\n{}'.format(type(a_range), a_range))

# create 5 rows of a_range, repeated 1 time each row
x = np.tile(a_range, (5, 1))
print('create 2d array `x` by spreading the range 5 times: {}, dimension: {}\n{}'.format(type(tile), tile.ndim, tile))

# transpose the tile
y = x.transpose()
print('transpose `x` and assign to y:\n', y)

# process x and y
xy = (((x-5)**2) + ((y-5) ** 2)) / 2
print('xy:\n', xy)

xy_scaled = xy/-(2 ** 2)
print('xy scaled:\n', xy_scaled)

# draw the surface
surface_x = np.exp(xy_scaled)
print('surface x: {}, dimension: {}\n{}'.format(type(surface_x), surface_x.ndim, surface_x))
v.surf(X=surface_x, opts=dict(colormap='Viridis'))

create array from int range 1 to 5: <class 'numpy.ndarray'>
[1 2 3 4 5]
create 2d array `x` by spreading the range 5 times: <class 'numpy.ndarray'>, dimension: 2
[[1 2 3 4 5]
 [1 2 3 4 5]
 [1 2 3 4 5]
 [1 2 3 4 5]
 [1 2 3 4 5]]
transpose `x` and assign to y:
 [[1 1 1 1 1]
 [2 2 2 2 2]
 [3 3 3 3 3]
 [4 4 4 4 4]
 [5 5 5 5 5]]
xy:
 [[16.  12.5 10.   8.5  8. ]
 [12.5  9.   6.5  5.   4.5]
 [10.   6.5  4.   2.5  2. ]
 [ 8.5  5.   2.5  1.   0.5]
 [ 8.   4.5  2.   0.5  0. ]]
xy scaled:
 [[-4.    -3.125 -2.5   -2.125 -2.   ]
 [-3.125 -2.25  -1.625 -1.25  -1.125]
 [-2.5   -1.625 -1.    -0.625 -0.5  ]
 [-2.125 -1.25  -0.625 -0.25  -0.125]
 [-2.    -1.125 -0.5   -0.125 -0.   ]]
surface x: <class 'numpy.ndarray'>, dimension: 2
[[0.01831564 0.04393693 0.082085   0.11943297 0.13533528]
 [0.04393693 0.10539922 0.19691168 0.2865048  0.32465247]
 [0.082085   0.19691168 0.36787944 0.53526143 0.60653066]
 [0.11943297 0.2865048  0.53526143 0.77880078 0.8824969 ]
 [0.13533528 0.32465247 0.60653066 0.8824969

'window_362900cd0bcee4'